## Preparing HDFS

Using magic

Create input folder on HDFS if not exists

Copy from data from local

In [2]:
val spark = org.apache.spark.sql.SparkSession.builder
        .master("local") 
        .appName("Spark CSV Reader")
        .getOrCreate;

2020-05-26 05:28:21,296 WARN  [Thread-4] sql.SparkSession$Builder (Logging.scala:logWarning(66)) - Using an existing SparkSession; some configuration may not take effect.


spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@7e6b2afc


In [3]:
import org.apache.spark.sql.functions._

import org.apache.spark.sql.functions._


In [4]:
! pwd
! hadoop fs -mkdir -p  /tmp/rs_input
! hadoop fs -put   -p  ./../data-raw/Melbourne_housing_FULL.csv             /tmp/rs_input/raw.csv
! hadoop fs -ls        /tmp/rs_input/
!hadoop fs -cat /tmp/rs_input/raw.csv | wc -l

/home/sandpit/big-data-realestate/scripts

put: `/tmp/rs_input/raw.csv': File exists


Found 1 items


-rw-r--r--   1 root root    5018236 2020-05-15 05:20 /tmp/rs_input/raw.csv


34858



In [5]:
//load raw into df
val df_raw = spark
    .read
    .format("csv")
    .option("header", "true")
    .load("hdfs://localhost:9000/tmp/rs_input/raw.csv")

df_raw: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


### Data Exploration

In [6]:
val df_raw = spark.read.format("csv").option("header", "true").load("hdfs://localhost:9000/tmp/rs_input/raw.csv")

df_raw: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


## Analysis

### Print schema:

In [7]:
df_raw.printSchema()

root
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Car: string (nullable = true)
 |-- Landsize: string (nullable = true)
 |-- BuildingArea: string (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)



In [8]:
df_raw.columns

res1: Array[String] = Array(Suburb, Address, Rooms, Type, Price, Method, SellerG, Date, Distance, Postcode, Bedroom2, Bathroom, Car, Landsize, BuildingArea, YearBuilt, CouncilArea, Lattitude, Longtitude, Regionname, Propertycount)


### Show column types:

In [9]:
df_raw.dtypes

res2: Array[(String, String)] = Array((Suburb,StringType), (Address,StringType), (Rooms,StringType), (Type,StringType), (Price,StringType), (Method,StringType), (SellerG,StringType), (Date,StringType), (Distance,StringType), (Postcode,StringType), (Bedroom2,StringType), (Bathroom,StringType), (Car,StringType), (Landsize,StringType), (BuildingArea,StringType), (YearBuilt,StringType), (CouncilArea,StringType), (Lattitude,StringType), (Longtitude,StringType), (Regionname,StringType), (Propertycount,StringType))


All features seem to be stored as string therefore numerical values will need to be converted to Integer and Float accordingly.
In addition, the categorical variables of Type and Method will need to be converted to integers as factors.

## Display the rows:

In [10]:
df_raw.select("Suburb","Address","POSTCODE","SUBURB","TYPE","METHOD","SELLERG","DATE","COUNCILAREA","REGIONNAME","YEARBUILT").show()

+----------+-------------------+--------+----------+----+------+-------+---------+------------------+--------------------+---------+
|    Suburb|            Address|POSTCODE|    SUBURB|TYPE|METHOD|SELLERG|     DATE|       COUNCILAREA|          REGIONNAME|YEARBUILT|
+----------+-------------------+--------+----------+----+------+-------+---------+------------------+--------------------+---------+
|Abbotsford|      68 Studley St|    3067|Abbotsford|   h|    SS| Jellis|3/09/2016|Yarra City Council|Northern Metropol...|     null|
|Abbotsford|       85 Turner St|    3067|Abbotsford|   h|     S| Biggin|3/12/2016|Yarra City Council|Northern Metropol...|     null|
|Abbotsford|    25 Bloomburg St|    3067|Abbotsford|   h|     S| Biggin|4/02/2016|Yarra City Council|Northern Metropol...|     1900|
|Abbotsford| 18/659 Victoria St|    3067|Abbotsford|   u|    VB| Rounds|4/02/2016|Yarra City Council|Northern Metropol...|     null|
|Abbotsford|       5 Charles St|    3067|Abbotsford|   h|    SP| Bigg

In [11]:
df_raw.select("Landsize","BuildingArea","YearBuilt","CouncilArea","Lattitude","Longtitude","Regionname","Propertycount").show(10)

+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|Landsize|BuildingArea|YearBuilt|       CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|     126|        null|     null|Yarra City Council| -37.8014|  144.9958|Northern Metropol...|         4019|
|     202|        null|     null|Yarra City Council| -37.7996|  144.9984|Northern Metropol...|         4019|
|     156|          79|     1900|Yarra City Council| -37.8079|  144.9934|Northern Metropol...|         4019|
|       0|        null|     null|Yarra City Council| -37.8114|  145.0116|Northern Metropol...|         4019|
|     134|         150|     1900|Yarra City Council| -37.8093|  144.9944|Northern Metropol...|         4019|
|      94|        null|     null|Yarra City Council| -37.7969|  144.9969|Northern Metropol...|         4019|
|     120|         

The data seems to be relatively clean however further exploration is required.

## EDA using python



#### Change type of columns "Price", "DistanceFromCBD" & "Landsize" to Double, "Rooms", "Bathroom", "Car" to Int

In [12]:
var df_anl = df_raw.withColumnRenamed("Method","MethodOfSale")
    .withColumnRenamed("Distance","DistanceFromCBD")
    .withColumnRenamed("Type","PropertyType")
    .withColumnRenamed("Lattitude","Latitude")
    

df_anl: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


In [13]:
df_anl = df_anl.withColumn("Price",col("Price").cast("Double"))
    .withColumn("Rooms",col("Rooms").cast("Int"))
    .withColumn("DistanceFromCBD",col("DistanceFromCBD").cast("Double"))
    .withColumn("Bathroom",col("Bathroom").cast("Int"))
    .withColumn("Car",col("Car").cast("Int"))
    .withColumn("Landsize",col("Landsize").cast("Double"))
    .withColumn("Latitude",col("Latitude").cast("String"))
    .withColumn("Longtitude",col("Longtitude").cast("String"))


df_anl: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


Share a dataset by registering it as a view. 

In [14]:
df_anl.toDF.createOrReplaceTempView("df_py_raw")

In [15]:
%%python
df_raw_p = spark.sql("select * from df_py_raw")


In [16]:
%%python
df_raw_p

DataFrame[Suburb: string, Address: string, Rooms: int, PropertyType: string, Price: double, MethodOfSale: string, SellerG: string, Date: string, DistanceFromCBD: double, Postcode: string, Bedroom2: string, Bathroom: int, Car: int, Landsize: double, BuildingArea: string, YearBuilt: string, CouncilArea: string, Latitude: string, Longtitude: string, Regionname: string, Propertycount: string]

In [17]:
! pip install matplotlib
! pip install pandas

In [18]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

categories=("A","B","C")
data = {
    
    'days': np.random.randint(120,size=100),
    'category': np.random.choice(categories,100),
    'value': 100.0*np.random.random_sample(100)
}
df=pd.DataFrame(data)
number_of_days=42
df_filtered = df.loc[df['days']<number_of_days]
df_filtered[["category","value"]].boxplot(by="category",return_type='axes')
plt.show()

In [27]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline


plt.clf()
plt.plot(range(10), label="a")
plt.plot(range(4, 14), label="b")
plt.plot(range(12, 0, -1), label="c")
plt.legend()
plt.show()

SyntaxError: ('invalid syntax', ('python cell', 3, 1, '%matplotlib inline\n'))

In [28]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

df_raw_pd = df_raw_p.toPandas() 
print(df_raw_pd.columns)

Index(['Suburb', 'Address', 'Rooms', 'PropertyType', 'Price', 'MethodOfSale',
       'SellerG', 'Date', 'DistanceFromCBD', 'Postcode', 'Bedroom2',
       'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt',
       'CouncilArea', 'Latitude', 'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')


In [82]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

df_raw_pd = df_raw_p.toPandas() 
print(df_raw_pd.head())

       Suburb             Address  ...             Regionname Propertycount
0  Abbotsford       68 Studley St  ...  Northern Metropolitan          4019
1  Abbotsford        85 Turner St  ...  Northern Metropolitan          4019
2  Abbotsford     25 Bloomburg St  ...  Northern Metropolitan          4019
3  Abbotsford  18/659 Victoria St  ...  Northern Metropolitan          4019
4  Abbotsford        5 Charles St  ...  Northern Metropolitan          4019

[5 rows x 21 columns]


In [ ]:
%%python
df_raw_pd.boxplot(column ='Price', by = 'PropertyType')

In [85]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

df_raw_pd = df_raw_p.toPandas() 
print(df_raw_pd.head())


       Suburb             Address  ...             Regionname Propertycount
0  Abbotsford       68 Studley St  ...  Northern Metropolitan          4019
1  Abbotsford        85 Turner St  ...  Northern Metropolitan          4019
2  Abbotsford     25 Bloomburg St  ...  Northern Metropolitan          4019
3  Abbotsford  18/659 Victoria St  ...  Northern Metropolitan          4019
4  Abbotsford        5 Charles St  ...  Northern Metropolitan          4019

[5 rows x 21 columns]


### Identify and remove outliers

In [31]:
%%python
print(type(df_raw_pd) )

<class 'pandas.core.frame.DataFrame'>


In [41]:
%%python
print(type(df_raw_pd) )
print(df_raw_pd.shape[0])
print(df_raw_pd.Price)

<class 'pandas.core.frame.DataFrame'>
34857
0              NaN
1        1480000.0
2        1035000.0
3              NaN
4        1465000.0
           ...    
34852    1480000.0
34853     888000.0
34854     705000.0
34855    1140000.0
34856    1020000.0
Name: Price, Length: 34857, dtype: float64


In [47]:
%%python

# filter price outliers
price_Q1 = df_raw_pd['Price'].quantile(0.25)
price_Q3 = df_raw_pd['Price'].quantile(0.75)
price_IQR = price_Q3 - price_Q1  
print(price_IQR)
price_filter = (df_raw_pd.Price >= price_Q1 - 1.5 * price_IQR) & (df_raw_pd.Price <= price_Q3 + 1.5 *price_IQR)

print(df_raw_pd.loc[price_filter].shape[0])
print(df_raw_pd.loc[price_filter])

660000.0
25969
           Suburb           Address  ...             Regionname Propertycount
1      Abbotsford      85 Turner St  ...  Northern Metropolitan          4019
2      Abbotsford   25 Bloomburg St  ...  Northern Metropolitan          4019
4      Abbotsford      5 Charles St  ...  Northern Metropolitan          4019
5      Abbotsford  40 Federation La  ...  Northern Metropolitan          4019
6      Abbotsford       55a Park St  ...  Northern Metropolitan          4019
...           ...               ...  ...                    ...           ...
34852  Yarraville       13 Burns St  ...   Western Metropolitan          6543
34853  Yarraville     29A Murray St  ...   Western Metropolitan          6543
34854  Yarraville    147A Severn St  ...   Western Metropolitan          6543
34855  Yarraville  12/37 Stephen St  ...   Western Metropolitan          6543
34856  Yarraville  3 Tarrengower St  ...   Western Metropolitan          6543

[25969 rows x 21 columns]


In [89]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

df_raw_pd.plot(kind='line',x='DistanceFromCBD',y='Price',color='red')
plt.show()


In [91]:
%%python
from matplotlib import pyplot as plt
import pandas as pd

df_raw_pd[["Price","PropertyType"]].boxplot( by="PropertyType", return_type='axes')
plt.show()

In [92]:
%%python
from matplotlib import pyplot as plt

plt.plot([1, 2, 3, 4])
plt.ylabel('some numbers')
plt.show()


In [91]:
%%python
'exec(%matplotlib inline)'
from matplotlib import pyplot as plt
import pandas as pd
# df.hist(column='session_duration_seconds')
#df_raw_p.select("Price").show()
df_raw_pd = df_raw_p.show().toPandas() # pandas.core.frame.DataFrame'>
df_raw_pd[:2].plot()
plt.show()
#df_raw_pd.plot(kind='scatter',x='Distance',y='Price',color='red')
#plt.show()
#df_raw_p.select("Price").show()
#df_raw_pd["Price"]
#df_raw_pd.show()
#plt.plot([1, 2, 3, 4])
#plt.ylabel('some numbers')
#plt.show()

+----------+-------------------+-----+----+-------+------+-------+---------+--------+--------+--------+--------+----+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|    Suburb|            Address|Rooms|Type|  Price|Method|SellerG|     Date|Distance|Postcode|Bedroom2|Bathroom| Car|Landsize|BuildingArea|YearBuilt|       CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+----------+-------------------+-----+----+-------+------+-------+---------+--------+--------+--------+--------+----+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|Abbotsford|      68 Studley St|    2|   h|   null|    SS| Jellis|3/09/2016|     2.5|    3067|       2|       1|   1|     126|        null|     null|Yarra City Council| -37.8014|  144.9958|Northern Metropol...|         4019|
|Abbotsford|       85 Turner St|    2|   h|1480000|     S| Biggin|3/12/2016|     2.5|    3067|      

AttributeError: ("'NoneType' object has no attribute 'toPandas'",)

## Descriptive Statistics:

In [ ]:
df_raw.describe().select("summary",
                        "Suburb",
                        "Address",
                        "Rooms",
                        "Type",
                        "Price",
                        "Method",
                        "SellerG",
                        "Date").show()

In [ ]:
df_raw.describe().select("summary",
                        "Distance",
                        "Postcode",
                        "Bedroom2",
                        "Bathroom",
                        "Car",
                        "Landsize").show()

In [ ]:
df_raw.describe().select("summary",
                        "BuildingArea",
                        "YearBuilt",
                        "CouncilArea",
                        "Lattitude",
                        "Longtitude",
                        "Regionname").show()

In [ ]:
df_raw.describe().select("summary",
                         "Propertycount").show()

Based on the above summary statistics we can see the following:
* The count of a column less the rows in the dataset, display the null values.
* The Address will need to be stripped down to its Street Name and Street Type. This is to  

### Correlation:

Assess the correlation between the Price and the other features to better understand their relationship and importance.

In [ ]:
// Import Correlation Library
import org.apache.spark.sql.functions.corr

In [ ]:
df_raw.select(corr("Rooms","Price")).show()
df_raw.select(corr("Distance","Price")).show()
df_raw.select(corr("Postcode","Price")).show()
df_raw.select(corr("Bedroom2","Price")).show()
df_raw.select(corr("Bathroom","Price")).show()
df_raw.select(corr("Car","Price")).show()
df_raw.select(corr("Landsize","Price")).show()
df_raw.select(corr("BuildingArea","Price")).show()
df_raw.select(corr("YearBuilt","Price")).show()
df_raw.select(corr("Lattitude","Price")).show()
df_raw.select(corr("Longtitude","Price")).show()
df_raw.select(corr("Propertycount","Price")).show()

Landsize having very minimal correlation with price does not align with expectations therefore further analysis is required.

We will compare the correlation of Price with Landsize based on the Property type.

- br - bedroom(s);
- h  - house,cottage,villa, semi,terrace;
- u  - unit, duplex;
- t  - townhouse;
- dev site - development site;
- o res - other residential

In [ ]:
// Correlation of Landsize and Price for Houses
df_raw.where($"Type" === "h").select(corr("Landsize","Price")).show()

In [ ]:
// Correlation of Landsize and Price for Units
df_raw.where($"Type" === "u").select(corr("Landsize","Price")).show()

In [ ]:
// Correlation of Landsize and Price for Townhouses
df_raw.where($"Type" === "t").select(corr("Landsize","Price")).show()

In [ ]:
// Create a new DataFrame with Price per SQM
val df_landprice = df_raw.withColumn("PriceperSQM", col("Price") / col("Landsize"))

// Assess Correlation of Price with Price per SQM
df_landprice.select(corr("Price","PriceperSQM")).show()

As expected, the Price vs PriceperSQM correlation is higher at ~11%, however this is due to price being included and compared against itself.

Therefore, it seems that Landsize on its own doesn't seem to have a significant correlation. However, correlation changes depending on the type of property this is. With a townhouse having the highest positive correlation of 0.096 or 9.6%.

As a result we conclude on keeping the LandSize.

In [ ]:
df_raw.select("Lattitude").distinct.count()

In [ ]:
df_raw.select("Longtitude").distinct.count()

#### Based on the preliminary analysis above, the features we have identified as important for the future model are:

* Address
* Suburb
* Date
* Price
* Method
* Type
* Distance
* Rooms
* Bathroom
* Car
* Landsize
* Lattitude
* Longtitude

The excluded features are:

* SellerG
* Postcode
* Bedroom2
* BuildingArea
* YearBuilt
* CouncilArea
* Regionname
* Propertycount


Therefore we will continue our analysis on the following.

### Categorical Attributes

#### Address

In [ ]:
df_raw.filter("Address IS NULL").count()

In [ ]:
df_raw.select("Address").distinct.show()

As seen above based on the complexity of Addresses, it would be useful to reduce them to a Street Name and Type in order to create categories within suburbs based on a street.

#### Suburb

In [ ]:
df_raw.select("Suburb").distinct.show()

In [ ]:
df_raw.select("Suburb").distinct.count()

The Suburbs generally seem to correct what will need to done is:
* Capitalise the first letter of the suburb names
* Also the North/West/South/East suffixes to suburbs will be left, as they provide a more accurate location within a suburb.

#### Date

In [ ]:
val dates = df_raw.select("Date").distinct()

In [ ]:
dates.count()

In [ ]:
df_raw.filter("Date IS NULL").count()

In [ ]:
dates.show(80)

#### Method

In [ ]:
df_raw.select("Method").distinct.show()

#### Null values  

In [ ]:
df_raw.filter("Method IS NULL").count()

#### Type 
#### Distinct values 

In [ ]:
df_raw.select("Type").distinct.show()

In [ ]:
df_raw = df_result.withColumn("Type", initcap(col("Type")))
df_raw.select("Type").distinct.show()

#### 9. Regionname

In [38]:
df_raw.select("Regionname").distinct.show()

+--------------------+
|          Regionname|
+--------------------+
|South-Eastern Met...|
|Western Metropolitan|
|Eastern Metropolitan|
|    Eastern Victoria|
|                #N/A|
|   Northern Victoria|
|Northern Metropol...|
|Southern Metropol...|
|    Western Victoria|
+--------------------+



In [39]:
df_raw.filter("Regionname IS NULL").count()

res7: Long = 0


# Wrangling

### The cleansing process based on the above findings has been completed in a separate notebook.

# Secondary Analysis

Perform a secondary analysis on the clean dataset to compare it with the original.

In [41]:
! hadoop fs -mkdir -p  /tmp/output
! hadoop fs -put   -p  ./../data-clean/*.csv             /tmp/output

In [50]:
// Load Clean Dataset into a DataFrame from HDFS after wrangling is completed
val df_clean = spark
    .read
    .format("csv")
    .option("header", "true")
    .load("hdfs://localhost:9000/tmp/output/*.csv")

df_clean: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [51]:
// Count the rows within the imported file
df_clean.count()

res8: Long = 15728


In [52]:
df_clean.printSchema()

root
 |-- Price: string (nullable = true)
 |-- MethodOfSale: string (nullable = true)
 |-- PropertyType: string (nullable = true)
 |-- DistanceFromCBD: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Car: string (nullable = true)
 |-- Landsize: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- StreetName: string (nullable = true)



In [53]:
df_clean.select("Price", 
                "MethodOfSale", 
                "PropertyType", 
                "DistanceFromCBD", 
                "Rooms", 
                "Bathroom", 
                "Car", 
                "Landsize", 
                "Latitude", 
                "Longtitude", 
                "Suburb", 
                "Date", 
                "StreetName").show()

+---------+------------+------------+---------------+-----+--------+---+--------+--------+----------+----------+----------+-------------+
|    Price|MethodOfSale|PropertyType|DistanceFromCBD|Rooms|Bathroom|Car|Landsize|Latitude|Longtitude|    Suburb|      Date|   StreetName|
+---------+------------+------------+---------------+-----+--------+---+--------+--------+----------+----------+----------+-------------+
|1480000.0|           1|           1|            2.5|    2|       1|  1|   202.0|-37.7996|  144.9984|Abbotsford| 3/12/2016|    Turner St|
|1035000.0|           1|           1|            2.5|    2|       1|  0|   156.0|-37.8079|  144.9934|Abbotsford| 4/02/2016| Bloomburg St|
|1465000.0|           2|           1|            2.5|    3|       2|  0|   134.0|-37.8093|  144.9944|Abbotsford| 4/03/2017|   Charles St|
| 850000.0|           3|           1|            2.5|    3|       2|  1|    94.0|-37.7969|  144.9969|Abbotsford| 4/03/2017|Federation La|
|1600000.0|           6|          

### Descriptive Statistics:

In [54]:
df_clean.describe().select("Summary", 
                           "Price", 
                           "MethodOfSale", 
                           "PropertyType", 
                           "DistanceFromCBD", 
                           "Rooms", 
                           "Bathroom").show()

+-------+-----------------+------------------+------------------+------------------+------------------+------------------+
|Summary|            Price|      MethodOfSale|      PropertyType|   DistanceFromCBD|             Rooms|          Bathroom|
+-------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|            15728|             15728|             15728|             15728|             15728|             15728|
|   mean|1150668.961851475|1.8519201424211598|1.2367115971515767|11.743203204475906| 3.187817904374364|1.6276068158697863|
| stddev|663018.9561677927|1.5641294862942188|0.5723389425747235| 6.700598921434572|0.8816674199865697|0.7148157821249159|
|    min|           1.12E7|                 1|                 1|               0.0|                 1|                 0|
|    max|         999999.0|                 8|                 3|               9.9|                 8|                 9|
+-------+-------

In [55]:
df_clean.describe().select("summary", 
                           "Car",
                           "Landsize", 
                           "Latitude", 
                           "Longtitude", 
                           "Suburb", 
                           "Date", 
                           "StreetName").show()

+-------+------------------+------------------+-------------------+-------------------+----------+---------+-----------+
|summary|               Car|          Landsize|           Latitude|         Longtitude|    Suburb|     Date| StreetName|
+-------+------------------+------------------+-------------------+-------------------+----------+---------+-----------+
|  count|             15728|             15728|              15728|              15728|     15728|    15728|      15728|
|   mean|1.7699008138351984| 668.6406408952187| -37.80422331192775|  144.9974195784588|      null|     null|       null|
| stddev|1.0185620996735723|4008.2209550592024|0.09275044706405701|0.12243663700518545|      null|     null|       null|
|    min|                 0|             100.0|          -37.39946|          144.42379|Abbotsford|1/07/2017|Aanensen Ct|
|    max|                 9|             999.0|          -38.19043|          145.52635|Yarraville|9/12/2017|Zurzolo Tce|
+-------+------------------+----

#### Correlation of the remainder varibles with Price

In [56]:
// Correlation now will include the categorical variables which where converted to factor.
df_clean.select(corr("MethodOfSale","Price")).show()
df_clean.select(corr("PropertyType","Price")).show()
df_clean.select(corr("DistanceFromCBD","Price")).show()
df_clean.select(corr("Rooms","Price")).show()
df_clean.select(corr("Bathroom","Price")).show()
df_clean.select(corr("Car","Price")).show()
df_clean.select(corr("Landsize","Price")).show()
df_clean.select(corr("Latitude","Price")).show()
df_clean.select(corr("Longtitude","Price")).show()

+-------------------------+
|corr(MethodOfSale, Price)|
+-------------------------+
|      0.09999735389602706|
+-------------------------+

+-------------------------+
|corr(PropertyType, Price)|
+-------------------------+
|     -0.19834317580327496|
+-------------------------+

+----------------------------+
|corr(DistanceFromCBD, Price)|
+----------------------------+
|         -0.3067923420146305|
+----------------------------+

+------------------+
|corr(Rooms, Price)|
+------------------+
|0.3891959982715667|
+------------------+

+---------------------+
|corr(Bathroom, Price)|
+---------------------+
|   0.4070422568142487|
+---------------------+

+------------------+
|  corr(Car, Price)|
+------------------+
|0.1508691880664582|
+------------------+

+---------------------+
|corr(Landsize, Price)|
+---------------------+
| 0.020547866303955897|
+---------------------+

+---------------------+
|corr(Latitude, Price)|
+---------------------+
|  -0.2523660525744002|
+-----------

In [68]:
df_clean

res15: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [59]:
df_clean.toDF.createOrReplaceTempView("df_py_clean")

In [74]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

df_clean_p = spark.sql("select * from df_py_clean")
df_clean_pd = df_clean_p.toPandas()
print(df_clean_pd.Price.quantile(0.25)) 

TypeError: ("can't multiply sequence by non-int of type 'float'",)

2020-05-26 11:38:18,847 WARN  [dispatcher-event-loop-2] spark.HeartbeatReceiver (Logging.scala:logWarning(66)) - Removing executor driver with no recent heartbeats: 3013031 ms exceeds timeout 120000 ms
2020-05-26 11:38:19,778 ERROR [dispatcher-event-loop-2] scheduler.TaskSchedulerImpl (Logging.scala:logError(70)) - Lost executor driver on localhost: Executor heartbeat timed out after 3013031 ms
2020-05-26 11:38:25,242 WARN  [kill-executor-thread] spark.SparkContext (Logging.scala:logWarning(66)) - Killing executors is not supported by current scheduler.
2020-05-26 11:38:25,307 WARN  [dispatcher-event-loop-0] storage.BlockManagerMasterEndpoint (Logging.scala:logWarning(66)) - No more replicas available for broadcast_10_piece0 !
2020-05-26 11:38:25,308 WARN  [dispatcher-event-loop-0] storage.BlockManagerMasterEndpoint (Logging.scala:logWarning(66)) - No more replicas available for broadcast_57_piece0 !


In [72]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

print(df_clean_pd.Price)

# filter price outliers
price_Q1 = df_clean_pd['Price'].quantile(0.25)
price_Q3 = df_clean_pd['Price'].quantile(0.75)
price_IQR = price_Q3 - price_Q1  
print(price_IQR)


0        1480000.0
1        1035000.0
2        1465000.0
3         850000.0
4        1600000.0
           ...    
15723     570000.0
15724    1480000.0
15725     888000.0
15726     705000.0
15727    1020000.0
Name: Price, Length: 15728, dtype: object


TypeError: ("can't multiply sequence by non-int of type 'float'",)

In [70]:
%%python
from matplotlib import pyplot as plt
import pandas as pd
from IPython.display import display

# filter price outliers
price_Q1 = df_clean_pd.Price.quantile(0.25)
price_Q3 = df_clean_pd.Price.quantile(0.75)
price_IQR = price_Q3 - price_Q1  
print(price_IQR)
price_filter = (df_clean_pd.Price >= price_Q1 - 1.5 * price_IQR) & (df_clean_pd.Price <= price_Q3 + 1.5 *price_IQR)

print(df_clean_pd.loc[price_filter].shape[0])
print(df_clean_pd.loc[price_filter])

TypeError: ("can't multiply sequence by non-int of type 'float'",)

## References

Apache Spark (n.d.). _Spark Scala API (Scaladoc). Overview._ https://spark.apache.org/docs/latest/api/java/overview-summary.html

Apache Spark (n.d.). _Basic Statistic._ https://spark.apache.org/docs/latest/ml-statistics.html

Bahadoor N. (2020). _Spark Tutorials_ https://allaboutscala.com/big-data/spark/#dataframe-statistics-correlation

Databricks. (2020). _Introduction to DataFrames - Scala._  https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-scala.html 

Grimaldi E. (2018). _Pandas vs. Spark: how to handle dataframes (Part II.)_  https://towardsdatascience.com/python-pandas-vs-scala-how-to-handle-dataframes-part-ii-d3e5efe8287d 

